# **5. Entrenamiento** **del** **modelo** **clasificador** **de** **pasos** **de** **la** **receta**

En este módulo se entrena el modelo clasificador encargado de dividir una receta escrita como un único bloque de texto en pasos ordenados.

In [ ]:
!pip install requests beautifulsoup4 pandas
!pip install -q openai ipywidgets python-dotenv
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # autoriza en el popup

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import os
import json

El primer paso es la adaptación del dataset de entrenemiento preprocesado para poder utilizarlo en el entrenamiento del modelo clasificador. El primer paso es añadir a las recetas una entrada 'resumen' con el texto en crudo, que servirá de input junto con la ya existente 'pasos' para el entrenar el modelo.

In [ ]:
from pathlib import Path
# Cargar dataset preprocesado
contenido = Path("recetas.json").read_text(encoding="utf-8")
recetas_cargadas = json.loads(contenido)

In [ ]:
#Funcion para añadir resumen a la receta
import re
from typing import Dict, List, Any

def agregar_resumen_a_receta(receta: Dict[str, Any], clave_pasos: str = "pasos", clave_resumen: str = "resumen") -> Dict[str, Any]:
    """
    Añade receta[clave_resumen] uniendo receta[clave_pasos] (lista de strings)
    en una sola cadena, separadas por punto. Limpia puntuación final duplicada.

    - Si no hay pasos o no es una lista de strings, pone resumen = "".
    - Devuelve el propio diccionario (mutado) por conveniencia.
    """
    pasos = receta.get(clave_pasos) or []
    if not isinstance(pasos, list):
        receta[clave_resumen] = ""
        return receta

    limpios: List[str] = []
    for p in pasos:
        if not isinstance(p, str):
            continue
        s = p.strip()
        # quita puntuación final para evitar ".."
        s = re.sub(r"\s*([.;:!?…]+)\s*$", "", s)
        if s:
            limpios.append(s)

    # une con ". " y añade punto final si hay contenido
    resumen = ". ".join(limpios)
    if resumen:
        resumen += "."
    receta[clave_resumen] = resumen
    return receta

In [ ]:
for receta in recetas_cargadas:
  receta = agregar_resumen_a_receta(receta)

A modo de ejemplo se muestra la receta 'Pollo crujiente con salsa césar', con el resumen incorporado

In [ ]:
recetas_cargadas[1000]

{'titulo': 'Pollo crujiente con salsa césar',
 'url': 'https://recetasdecocina.elmundo.es/2012/08/pollo-crujiente-con-salsa-cesar.html',
 'ingredientes': ['frutos secos',
  'solomillo de pollo',
  'sal',
  'pimienta',
  'huevo',
  'aceite'],
 'pasos': ['Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños.',
  'Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados.',
  'Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'],
 'resumen': 'Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños. Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados. Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'}

El siguiente paso es el procesamiento de los 'pasos' de las recetas para obtener un input interpretable por el modelo clasificador. A cada oración del paso se le asigna una etiqueta que puede ser 'B' si la oración inicia un nuevo paso o 'I' si la oración esta contenida en un paso ya iniciado.

Para este procesamiento del dataset definimos las siguientes funciones:

- ***_rm_acc***: devuelve las cadenas de caracteres sin tildes/acentos.

- ***_build_norm_with_map***: genera una versión normalizada del texto (minúsculas, sin tildes y con espacios colapsados) y un mapa que liga cada carácter normalizado con su índice en el original(norm, idx_map).

- ***oraciones_con_offsets***: segmenta el texto por signos de puntuación (. ! ? …), salta espacios siguientes y retorna una lista de tuplas (inicio, fin, oración)

- ***_overlap***: calcula la longitud del solapamiento entre los intervalos semiabiertos, lo que permite al modelo decidir a que paso le corresponde cada oración (con qué paso queda mejor alineada cada oración).

- ***localizar_pasos_en_resumen***: busca cada paso (normalizado) dentro del resumen (normalizado), y devuelve sus offsets (start, end) en el texto original usando el mapa de índices.

- ***etiquetar_bi_por_oracion***: tiqueta cada oración como “B” cuando inicia o cambia de paso y como “I” cuando continúa el mismo.

- ***construir_oraciones_dataset_bi*** :genera el datasaet con las oraciones del resumen de la receta etiquetadas.

In [ ]:
!pip install -q transformers datasets accelerate scikit-learn


In [ ]:
import re, unicodedata, numpy as np, torch
from typing import Tuple
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer)
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


In [ ]:
def _rm_acc(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def _build_norm_with_map(text: str):
    """Normaliza (minúsculas, sin tildes, colapsa espacios) + mapa a índices del original."""
    norm_chars, idx_map = [], []
    last_space = False
    for i, ch in enumerate(text):
        base = _rm_acc(ch).lower()
        if base.isspace():
            if not last_space:
                norm_chars.append(" ")
                idx_map.append(i)
            last_space = True
        else:
            norm_chars.append(base)
            idx_map.append(i)
            last_space = False
    norm = " ".join("".join(norm_chars).strip().split())
    idx_map = idx_map[:len(norm)]
    return norm, idx_map

def oraciones_con_offsets(texto: str):
    """Divide por . ! ? …  y devuelve [(start,end,oración)] (end exclusivo)."""
    bounds, start = [], 0
    for m in re.finditer(r"[.!?…]+", texto):
        end = m.end()
        if end > start:
            bounds.append((start, end))
        nxt = end
        while nxt < len(texto) and texto[nxt].isspace():
            nxt += 1
        start = nxt
    if start < len(texto):
        bounds.append((start, len(texto)))
    return [(a,b,texto[a:b]) for a,b in bounds]

def _overlap(a1,b1,a2,b2):
    return max(0, min(b1,b2) - max(a1,a2))

def localizar_pasos_en_resumen(resumen: str, pasos: List[str]):
    """Encuentra pasos en el resumen con búsqueda robusta (normalizada)."""
    offs, cursor_norm = [], 0
    H, Hmap = _build_norm_with_map(resumen)
    for p in (pasos or []):
        p = (p or "").strip()
        if not p:
            continue
        N, _ = _build_norm_with_map(p)
        if not N:
            continue
        pos = H.find(N, cursor_norm)
        if pos == -1:
            pos = H.find(N)
            if pos == -1:
                continue
        try:
            s = Hmap[pos]
            e = Hmap[pos + len(N) - 1] + 1
            offs.append((s, e))
            cursor_norm = pos + len(N)
        except Exception:
            continue
    return offs

def etiquetar_bi_por_oracion(resumen: str, pasos: List[str]) -> List[str]:
    """
    Etiqueta BI por oración:
      - 'B' si empieza un paso (o cambia de paso),
      - 'I' si continúa el mismo paso.
    Si una oración no solapa con ningún paso, continúa el actual (o abre 'B' si es la primera).
    """
    sents = oraciones_con_offsets(resumen)
    steps = localizar_pasos_en_resumen(resumen, pasos)
    if not sents:
        return []
    asign = []
    for (sa, sb, _) in sents:
        best_k, best_ov = None, 0
        for k, (pa, pb) in enumerate(steps):
            ov = _overlap(sa, sb, pa, pb)
            if ov > best_ov:
                best_k, best_ov = k, ov
        asign.append(best_k if best_ov > 0 else None)

    labels, prev_step = [], None
    for i, k in enumerate(asign):
        if k is None:
            k = prev_step if prev_step is not None else 0
        labels.append("B" if (prev_step is None or k != prev_step) else "I")
        prev_step = k
    return labels


In [ ]:
def construir_oraciones_dataset_bi(recetas: List[Dict[str,Any]], usar_contexto: bool = True):
    filas = []
    for r in recetas:
        resumen = (r.get("resumen") or "").strip()
        pasos   = r.get("pasos") or []
        if not resumen:
            continue
        sents = oraciones_con_offsets(resumen)
        if not sents:
            continue
        labels = etiquetar_bi_por_oracion(resumen, pasos)
        if len(labels) != len(sents):
            labels = ["I"] * len(sents)  # fallback seguro

        for i, (_,_,sent) in enumerate(sents):
            row = {"text": sent, "label": labels[i]}
            if usar_contexto:
                row["prev"] = sents[i-1][2] if i>0 else ""
                row["next"] = sents[i+1][2] if i < len(sents)-1 else ""
            filas.append(row)
    return filas


El dataset de entrenamiento del modelo clasificador se genera mediante un pipeline que incluye la tokenización y etiquetación del texto 'resumen' de cada receta. Par ello se utilizan las funciones ***formatear_entrada***, ***tokenize_with_labels*** y ***build_datasets_desde_recetas***, así como la previamente descrita ***construir_oraciones_dataset_bi***.

In [ ]:
MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased'   # o ruta local a tu modelo/tokenizer
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)

label_list = ["B","I"]
label2id   = {"B":0, "I":1}
id2label   = {0:"B", 1:"I"}

def formatear_entrada(ex):
    if "prev" in ex and "next" in ex:
        sep = tokenizer.sep_token or "</s>"
        ex["input_text"] = f"{ex['prev'].strip()} {sep} {ex['text'].strip()} {sep} {ex['next'].strip()}".strip()
    else:
        ex["input_text"] = ex["text"]
    ex["labels"] = label2id[ex["label"]]
    return ex

def tokenize_with_labels(batch):
    out = tokenizer(batch["input_text"], truncation=True, max_length=256)
    out["labels"] = batch["labels"]          # <- preserva labels para Trainer
    return out

def build_datasets_desde_recetas(recetas: List[Dict[str,Any]], usar_contexto=True):
    filas = construir_oraciones_dataset_bi(recetas, usar_contexto=usar_contexto)
    train_rows, tmp_rows = train_test_split(filas, test_size=0.2, random_state=42, shuffle=True)
    val_rows,   test_rows= train_test_split(tmp_rows, test_size=0.5, random_state=42, shuffle=True)

    ds = DatasetDict({
        "train": Dataset.from_list(train_rows),
        "validation": Dataset.from_list(val_rows),
        "test": Dataset.from_list(test_rows)
    })
    ds = ds.map(formatear_entrada)
    ds_tok = ds.map(tokenize_with_labels, batched=True, remove_columns=ds["train"].column_names)
    collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return ds_tok, collator


Definimos la función ***compute_metrics*** para evaluar el modelo con las métricas estandar, y la función ***entrenar_clf_bi*** para definir los parámetros de entrenamiento del modelo.

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc   = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

def entrenar_clf_bi(ds_tok: DatasetDict, collator, output_dir="/content/bi_clf_sent", epochs=4, lr=2e-5):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=2, id2label=id2label, label2id=label2id
    )
    args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=lr,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=epochs,
        logging_steps=50,
        report_to="none",
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    # eval al final (también puedes llamar luego a trainer.evaluate(ds_tok["test"]))
    print(trainer.evaluate(ds_tok["validation"]))
    return trainer


Se definen dos funciones que se utilizarán en el pipeline de extracción de pasos a partir de la receta en crudo:

-  ***predecir_bi_por_oracion***, que toma el resumen de la receta, lo divide en oraciones las tokeniza, las pasa por el modelo y devuelve para cada oración si es inicio (B) o continuación (I) de un paso.

- ***reconstruir_pasos_desde_BI***: A partir de las etiquetas B/I y las oraciones, reconstruye los pasos completos del resumen concatenando oraciones hasta que aparece una nueva B.

In [ ]:
@torch.no_grad()
def predecir_bi_por_oracion(resumen: str, tokenizer, model, usar_contexto: bool = True):
    model.eval()
    sents = oraciones_con_offsets(resumen)
    if not sents:
        return []
    textos = []
    for i, (_,_,sent) in enumerate(sents):
        if usar_contexto:
            prev_txt = sents[i-1][2] if i>0 else ""
            next_txt = sents[i+1][2] if i < len(sents)-1 else ""
            sep = tokenizer.sep_token or "</s>"
            textos.append(f"{prev_txt.strip()} {sep} {sent.strip()} {sep} {next_txt.strip()}".strip())
        else:
            textos.append(sent)
    enc = tokenizer(textos, padding=True, truncation=True, max_length=256, return_tensors="pt")
    enc = {k: v.to(model.device) for k,v in enc.items()}
    logits = model(**enc).logits
    preds  = logits.argmax(-1).cpu().numpy().tolist()
    return [id2label[i] for i in preds]  # "B" o "I"

def reconstruir_pasos_desde_BI(resumen: str, etiquetas_bi: List[str]):
    sents = oraciones_con_offsets(resumen)
    assert len(sents) == len(etiquetas_bi)
    pasos_offsets, pasos_textos = [], []
    cur = None
    def cerrar():
        nonlocal cur
        if cur:
            a,b = cur
            pasos_offsets.append((a,b))
            pasos_textos.append(resumen[a:b])
        cur = None
    for (a,b,_), tag in zip(sents, etiquetas_bi):
        tag = (tag or "I").upper()
        if tag == "B" or cur is None:
            cerrar(); cur = [a,b]
        else:  # I
            cur[1] = b
    cerrar()
    return {"pasos": pasos_textos, "pasos_offsets": pasos_offsets}


Entrenamos el modelo:

In [ ]:
ds_tok, collator = build_datasets_desde_recetas(recetas_cargadas, usar_contexto=True)
trainer = entrenar_clf_bi(ds_tok, collator, output_dir="/content/bi_clf_sent", epochs=4, lr=2e-5)

modelo = trainer.model

Map:   0%|          | 0/7675 [00:00<?, ? examples/s]

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/7675 [00:00<?, ? examples/s]

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-543084996.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.580900
100,0.548600
150,0.550800
200,0.536300
250,0.507600
300,0.509200
350,0.445600
400,0.385000
450,0.444600
500,0.387200


{'eval_loss': 0.6461438536643982, 'eval_accuracy': 0.8039624608967675, 'eval_precision': 0.6305220883534136, 'eval_recall': 0.6205533596837944, 'eval_f1': 0.6254980079681275, 'eval_runtime': 5.1989, 'eval_samples_per_second': 184.463, 'eval_steps_per_second': 5.77, 'epoch': 4.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/TFM/Classifier_model")  # guarda pesos + config.json
tokenizer.save_pretrained("/content/drive/MyDrive/TFM/Classifier_model")

('/content/drive/MyDrive/TFM/Classifier_model/tokenizer_config.json',
 '/content/drive/MyDrive/TFM/Classifier_model/special_tokens_map.json',
 '/content/drive/MyDrive/TFM/Classifier_model/vocab.txt',
 '/content/drive/MyDrive/TFM/Classifier_model/added_tokens.json',
 '/content/drive/MyDrive/TFM/Classifier_model/tokenizer.json')

Probamos el modelo con nuevas recetas para analizar su rendimiento.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

LOAD_DIR = "/content/drive/MyDrive/TFM/Classifier_model"
tokenizer = AutoTokenizer.from_pretrained(LOAD_DIR)
modelo     = AutoModelForSequenceClassification.from_pretrained(LOAD_DIR)

In [ ]:
texto = ("Pon en la olla ajo. Mezclalo con la cebolla. Despues añade el tomate")
y_bi = predecir_bi_por_oracion(texto, tokenizer, modelo, usar_contexto=True)
seg  = reconstruir_pasos_desde_BI(texto, y_bi)
print("Etiquetas:", y_bi)            # p.ej. ['B','I','B']
print("Pasos:", seg["pasos"])        # subcadenas EXACTAS del resumen
print("Offsets:", seg["pasos_offsets"])

Etiquetas: ['B', 'I', 'B']
Pasos: ['Pon en la olla ajo. Mezclalo con la cebolla.', 'Despues añade el tomate']
Offsets: [(0, 44), (45, 68)]


In [ ]:
texto = ("Pon en la olla ajo. Dejalo reposar un minuto. Mezclalo con la cebolla. Despues añade el tomate")
y_bi = predecir_bi_por_oracion(texto, tokenizer, modelo, usar_contexto=True)
seg  = reconstruir_pasos_desde_BI(texto, y_bi)
print("Etiquetas:", y_bi)            # p.ej. ['B','I','B']
print("Pasos:", seg["pasos"])        # subcadenas EXACTAS del resumen
print("Offsets:", seg["pasos_offsets"])

Etiquetas: ['B', 'I', 'B', 'B']
Pasos: ['Pon en la olla ajo. Dejalo reposar un minuto.', 'Mezclalo con la cebolla.', 'Despues añade el tomate']
Offsets: [(0, 45), (46, 70), (71, 94)]


In [ ]:
texto = ("Pon en la olla ajo. Tras ello pon la cebolla. Despues añade el tomate")
y_bi = predecir_bi_por_oracion(texto, tokenizer, modelo, usar_contexto=True)
seg  = reconstruir_pasos_desde_BI(texto, y_bi)
print("Etiquetas:", y_bi)            # p.ej. ['B','I','B']
print("Pasos:", seg["pasos"])        # subcadenas EXACTAS del resumen
print("Offsets:", seg["pasos_offsets"])

Etiquetas: ['B', 'B', 'B']
Pasos: ['Pon en la olla ajo.', 'Tras ello pon la cebolla.', 'Despues añade el tomate']
Offsets: [(0, 19), (20, 45), (46, 69)]


In [ ]:
texto = ("Pon en la olla ajo. Y también pon la cebolla. Despues añade el tomate")
y_bi = predecir_bi_por_oracion(texto, tokenizer, modelo, usar_contexto=True)
seg  = reconstruir_pasos_desde_BI(texto, y_bi)
print("Etiquetas:", y_bi)            # p.ej. ['B','I','B']
print("Pasos:", seg["pasos"])        # subcadenas EXACTAS del resumen
print("Offsets:", seg["pasos_offsets"])

Etiquetas: ['B', 'I', 'B']
Pasos: ['Pon en la olla ajo. Y también pon la cebolla.', 'Despues añade el tomate']
Offsets: [(0, 45), (46, 69)]
